# Page Rank
## March 8th, 2022
### Overview: Implementing the Page Rank algorithm to rank websites, basketball players, and actors according to relevancy

In [1]:
import numpy as np
import networkx as nx
from scipy import linalg as la
from itertools import combinations

In [2]:
class DiGraph:
    """A class for representing directed graphs via their adjacency matrices.

    Attributes:
        (fill this out after completing DiGraph.__init__().)
    """
    # Problem 1
    def __init__(self, A, labels=None):
        """Modify A so that there are no sinks in the corresponding graph,
        then calculate Ahat. Save Ahat and the labels as attributes.

        Parameters:
            A ((n,n) ndarray): the adjacency matrix of a directed graph.
                A[i,j] is the weight of the edge from node j to node i.
            labels (list(str)): labels for the n nodes in the graph.
                If None, defaults to [0, 1, ..., n-1].
        """
        #save A and replace any zero columns with ones
        self.A = A
        for i,row in enumerate(self.A.T):
            if all([el==0 for el in row]):
                self.A.T[i] = np.ones( len(self.A.T[0]))
                
        #Divide each column by its sum
        self.A /= np.sum(A,axis=0)
        
        #saving labels
        if labels is None:
            self.labels = [str(j) for j in np.arange(len(A))]
        else:
            if len(labels) != len(A):
                raise ValueError("Number of labels does not match number of nodes")
            self.labels = labels
    
    # Problem 2
    def linsolve(self, epsilon=0.85):
        """Compute the PageRank vector using the linear system method.

        Parameters:
            epsilon (float): the damping factor, between 0 and 1.

        Returns:
            dict(str -> float): A dictionary mapping labels to PageRank values.
        """
        #n is number of nodes, create nxn identity matrix
        n = len(self.A)
        I = np.identity(n)
        
        #left hand side of linear equation, as defined
        l = I - epsilon*self.A
        #right hand side, as defined
        r = (1 - epsilon)*np.ones(n)/(n)
        
        #solve for p, create dict for vals
        p = la.solve(l,r)
        D = dict(zip(self.labels,p))
        return D

    # Problem 2
    def eigensolve(self, epsilon=0.85):
        """Compute the PageRank vector using the eigenvalue method.
        Normalize the resulting eigenvector so its entries sum to 1.

        Parameters:
            epsilon (float): the damping factor, between 0 and 1.

        Return:
            dict(str -> float): A dictionary mapping labels to PageRank values.
        """
        #use create n, E, and B as defined
        n = len(self.A)
        E = np.ones((n,n))
        B = epsilon*self.A + (1 - epsilon)*E/n
        
        #p is the eigenvector of B corresponding to the eigenvalue 1 
        p = la.eig(B)[1].T[0]
        
        #normalize p, create dict for vals
        p /= p.sum()
        print(p)
        D = dict(zip(self.labels,p))
        
        return D

    # Problem 2
    def itersolve(self, epsilon=0.85, maxiter=100, tol=1e-12):
        """Compute the PageRank vector using the iterative method.

        Parameters:
            epsilon (float): the damping factor, between 0 and 1.
            maxiter (int): the maximum number of iterations to compute.
            tol (float): the convergence tolerance.

        Return:
            dict(str -> float): A dictionary mapping labels to PageRank values.
        """
        #use n, init p1 and diff
        n = len(self.A)
        p1 = np.array([1./n for el in range(n)])
        diff = 1
        
        #init iter var and enter while loop to calc next p vals
        it = 0
        while it < maxiter and diff > tol:
            #calc p2
            p2 = (epsilon * self.A@p1) + (1 - epsilon)*np.ones(n)/n
            #norm of difference
            diff = la.norm(p1 - p2,ord=1)
            
            #iterate p1
            p1 = p2
            #inc iter
            it += 1
        
        #create dict for vals
        D = dict(zip(self.labels,p1))
        return D

In [4]:
# Problem 3
def get_ranks(d):
    """Construct a sorted list of labels based on the PageRank vector.

    Parameters:
        d (dict(str -> float)): a dictionary mapping labels to PageRank values.

    Returns:
        (list) the keys of d, sorted by PageRank value from greatest to least.
    """
    #get indices which sort values in descending order
    pairs = [(-val,key) for key,val in zip(d.keys(), d.values())]
    pairs.sort()

    return [key[1] for key in pairs]

In [5]:
# Problem 4
def rank_websites(filename="web_stanford.txt", epsilon=0.85):
    """Read the specified file and construct a graph where node j points to
    node i if webpage j has a hyperlink to webpage i. Use the DiGraph class
    and its itersolve() method to compute the PageRank values of the webpages,
    then rank them with get_ranks(). If two webpages have the same rank,
    resolve ties by listing the webpage with the larger ID number first.

    Each line of the file has the format
        a/b/c/d/e/f...
    meaning the webpage with ID 'a' has hyperlinks to the webpages with IDs
    'b', 'c', 'd', and so on.

    Parameters:
        filename (str): the file to read from.
        epsilon (float): the damping factor, between 0 and 1.

    Returns:
        (list(str)): The ranked list of webpage IDs.
    """
    #read the file and split
    with open(filename) as readfile:
        L = readfile.read().splitlines()
        
    #hold all labels (as ints) in a set
    labels = set()
    for line in L:
        for label in line.split("/"):
            labels.add(int(label))
    
    #change labels to be list of strings for iteration, but sort them in reverse order first
    labels = [str(a) for a in sorted(list(labels))[::-1]]
    
    #init a dict to map a label to a specific index
    inds = dict()
    for i, label in enumerate(labels):
        inds[label] = i
    
    #init a square matrix of length of labels
    A = np.zeros((len(labels),len(labels)))
    
    #for each line in the file
    for line in L:
        #get label and all its links
        label, slash, links = line.partition("/")
        
        #get the index of the label
        I = inds[label]
        
        #for each link, get its index and set the respective entry in the matrix to 1
        for link in links.split("/"):
            I2 = inds[link]
            A[I2][I] = 1
    
    #create graph
    Graph = DiGraph(A,labels)
    
    #do itersolve and return ranks
    return get_ranks(Graph.itersolve(epsilon))

In [6]:
rank_websites()[:20]

['98595',
 '32791',
 '28392',
 '77323',
 '92715',
 '26083',
 '130094',
 '99464',
 '12846',
 '106064',
 '332',
 '31328',
 '86049',
 '123900',
 '74923',
 '119538',
 '90571',
 '116900',
 '139197',
 '15672']

In [7]:
# Problem 5
def rank_ncaa_teams(filename, epsilon=0.85):
    """Read the specified file and construct a graph where node j points to
    node i with weight w if team j was defeated by team i in w games. Use the
    DiGraph class and its itersolve() method to compute the PageRank values of
    the teams, then rank them with get_ranks().

    Each line of the file has the format
        A,B
    meaning team A defeated team B.

    Parameters:
        filename (str): the name of the data file to read.
        epsilon (float): the damping factor, between 0 and 1.

    Returns:
        (list(str)): The ranked list of team names.
    """
    #read the file and split it up
    with open(filename) as readfile:
        goTeam = readfile.read().splitlines()
        
    #create a set to hold all the teams
    teamSet = set()
    for couple in goTeam[1:]:
        team1,comma,team2 = couple.partition(",")
        teamSet.add(team1)
        teamSet.add(team2)
    
    #make set into list to iterate through it
    teams = list(teamSet)
    
    #create square matrix of length of teams
    n = len(teamSet)
    A = np.zeros((n,n))
    
    #create a dictionary to map a team to a specific index
    inds = dict()
    for i,team in enumerate(teams):
        inds[team] = i
    
    #for every coupling of winners and losers
    for couple in goTeam[1:]:
        winner, comma, loser = couple.partition(",")
        
        #at the row of the index of the winner
        #at the column of the index of the loser         - inc edge value
        A[inds[winner]][inds[loser]] += 1
    
    #make into class object, do itersolve and ranking
    Graph = DiGraph(A,teams)
    return get_ranks(Graph.itersolve(epsilon))

In [8]:
rank_ncaa_teams('ncaa2010.csv')[:15]

['UConn',
 'Kentucky',
 'Louisville',
 'Notre Dame',
 'Florida',
 'BYU',
 "St. John's (NY)",
 'Kansas',
 'VCU',
 'Syracuse',
 'Pitt',
 'Ohio State',
 'UNC',
 'West Virginia',
 'Butler']

In [9]:
# Problem 6
def rank_actors(filename="top250movies.txt", epsilon=0.85):
    """Read the specified file and construct a graph where node a points to
    node b with weight w if actor a and actor b were in w movies together but
    actor b was listed first. Use NetworkX to compute the PageRank values of
    the actors, then rank them with get_ranks().

    Each line of the file has the format
        title/actor1/actor2/actor3/...
    meaning actor2 and actor3 should each have an edge pointing to actor1,
    and actor3 should have an edge pointing to actor2.
    """
    #init a graph
    DG = nx.DiGraph()
    
    #open that dang file
    with open(filename,encoding='utf-8') as readfile:
        #each line is a different movie
        for line in readfile:
            #get all the actors by stripping all whitespace, splitting by the slashes and skipping the movie title
            actors = line.strip().split('/')[1:]
            
            #for ALL 2-actor combinations 
            for a1, a2 in combinations(actors,2):
                
                #if there's already an edge between them, inc the weight
                if DG.has_edge(a2,a1):
                    DG[a2][a1]["weight"] += 1
                #ldfkjsldkfjohoij;l widfjdlk.fjv; d shfkvj,mmfkci mpdkvf uco ij.xufn ld, kku h,zrfkidr.d kiuj,fmgoir 8 dt ;lkj jt hl. bj im nvkkmn nlgkgn\g mtlk
                #otherwise start an edge with weight 1
                else:
                    DG.add_edge(a2,a1,weight=1)
    
    #return ranking of actors
    return get_ranks(nx.pagerank(DG,alpha=epsilon))

In [10]:
rank_actors()[:15]

['Leonardo DiCaprio',
 'Jamie Foxx',
 'Christoph Waltz',
 'Robert De Niro',
 'Al Pacino',
 'Tom Hanks',
 'Antonella Attili',
 'Brad Pitt',
 'Christian Bale',
 'Ben Kingsley',
 'Diahnne Abbott',
 'Liam Neeson',
 'Ralph Fiennes',
 'Matt Damon',
 'Tom Hardy']